In [30]:
from PIL import Image, UnidentifiedImageError
import os
import re
import sys
import concurrent
import torch
import json
from torchvision import transforms
from image_data_handler_test import TestImageDataHandler
from geo_model_tester import GeoModelTester
from wandb_downloader import WandbDownloader
from custom_image_dataset_test import CustomImageDatasetTest

sys.path.insert(0, '../')
from data_loader import resolve_env_variable, load_image_file_raw, get_image_files

sys.path.insert(0, './4_modeling')
from region_handler import RegionHandler

In [31]:
region_handler = RegionHandler()

test_data_path = "C:\\Users\\yutar\\Documents\\HSLU\\Paper_data\\yfcc4k"
print("Test data path: ", test_data_path)
image_path = os.path.join(test_data_path, "image_processed")
json_path = os.path.join(test_data_path, "jsons")

Test data path:  C:\Users\yutar\Documents\HSLU\Paper_data\yfcc4k


In [32]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    
    # Print the name of the GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Print the total and available memory
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    print(f"Total Memory: {total_memory:.2f} GB")

    allocated_memory = torch.cuda.memory_allocated(0) / 1e9  # Convert bytes to GB
    print(f"Allocated Memory: {allocated_memory:.2f} GB")

    cached_memory = torch.cuda.memory_reserved(0) / 1e9  # Convert bytes to GB
    print(f"Cached Memory: {cached_memory:.2f} GB")

    # Print other properties
    device_properties = torch.cuda.get_device_properties(0)
    print(f"CUDA Capability: {device_properties.major}.{device_properties.minor}")
    print(f"Multi-Processor Count: {device_properties.multi_processor_count}")
else:
    print("No GPU found.")

No GPU found.


In [33]:
# get all image files
image_files = [os.path.join(image_path, f) for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]
print("Number of image files: ", len(image_files))
json_files = [os.path.join(json_path, f) for f in os.listdir(json_path) if os.path.isfile(os.path.join(json_path, f))]
print("Number of json files: ", len(json_files))

Number of image files:  4536
Number of json files:  4536


In [34]:
# check if all image files have a corresponding json file and vice versa by comparing the file names without the extension
image_file_names = [os.path.splitext(os.path.basename(f))[0] for f in image_files]
json_file_names = [os.path.splitext(os.path.basename(f))[0] for f in json_files]
image_file_names = set(image_file_names)
json_file_names = set(json_file_names)

# check if all image files have a corresponding json file
missing_json_files = image_file_names - json_file_names
if len(missing_json_files) > 0:
    print("Missing json files: ", missing_json_files)

# check if all json files have a corresponding image file
missing_image_files = json_file_names - image_file_names
if len(missing_image_files) > 0:
    print("Missing image files: ", missing_image_files)

# remove image files without a corresponding json file
image_files = [f for f in image_files if os.path.splitext(os.path.basename(f))[0] not in missing_json_files]

# remove json files without a corresponding image file
json_files = [f for f in json_files if os.path.splitext(os.path.basename(f))[0] not in missing_image_files]

# create a list of tuples with the image and the data inside corresponding json file "coordinates":, "country_name", "country_code", "regions", "is_in_region" as keys
data = []
for image_file in image_files:
    image_file_name = os.path.splitext(os.path.basename(image_file))[0]
    json_file = [f for f in json_files if os.path.splitext(os.path.basename(f))[0] == image_file_name]
    if len(json_file) == 1:
        json_file = json_file[0]
        with open(json_file, "r", encoding="utf-8") as f:
            json_data = json.load(f)
            data.append((image_file, json_data))

# print first 5 entries
for i in range(5):
    print(data[i])


In [4]:

running_device = "colab_L4"
image_size = [80, 130]
data_augmentation = "base_augmentation" # or "base_augmentation", "full_augmentation_v2"
predict_coordinates=False
predict_regions=True

if running_device == "colab_L4":
    # Run unmapped images with low image resolution
    USE_MAPPED = False

elif running_device == "colab_A100":
    # Run mapped images with high image resolution
    image_size = [180, 320]
    NUMBER_OF_FILES = 79000

In [5]:
prediction_type = "regions" if predict_regions else ("coordinates" if predict_coordinates else "countries")

train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

preprocessing_config = { 'data_augmentation': data_augmentation, 'height': image_size[0], 'width': image_size[1], 'train_ratio': train_ratio, 'val_ratio': val_ratio, 'test_ratio': test_ratio }

base_transform = transforms.Compose([
          transforms.Resize((image_size[0], image_size[1])),
        ])
augmented_transform = None
final_transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

if data_augmentation == "full_augmentation_v2":
    base_transform = transforms.Compose([])
    augmented_transform = transforms.Compose([
        # Disabled because black bars really hurt the performance at this size (only for v2)
        # transforms.RandomPerspective(distortion_scale=0.75, p=0.5),  # Randomly apply perspective transformation
        transforms.RandomResizedCrop((image_size[0], image_size[1]), scale=(0.75, 1.0)),  # Randomly crop the image and resize it to the original size
        transforms.RandomRotation(10),          # Randomly rotate the image by up to 10 degrees, sadly also causes black borders
        transforms.ColorJitter(
            brightness=(0.5, 1.5),  # Randomly change brightness (lower limit to simulate night, upper limit for bright daylight)
            contrast=(0.5, 1.5),    # Randomly change contrast
            saturation=(0.5, 1.5),  # Randomly change saturation
            hue=(-0.1, 0.1)         # Randomly change hue
        )
    ])

In [6]:
project_name = "predicting-coordinates" if predict_coordinates else ("predicting-region" if predict_regions else "predicting-country")
sweep_metric_name = "Validation Distance (km)" if predict_coordinates or predict_regions else "Validation Accuracy Top 1"


In [7]:
wandb_downloader = WandbDownloader(entity="nlp_ls", project=f"dspro2-{project_name}", data_augmentation=data_augmentation, input_image_size=image_size)

file_names_to_download = [".pth", ".json"]

run_data = wandb_downloader.get_and_collect_best_runs(sweep_metric_name, file_names_to_download)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\yutar\.netrc
wandb: Currently logged in as: yutarosigrist (nlp-yuuta). Use `wandb login --relogin` to force relogin


No matching runs found.
